# Tutorial 3: Learning

In [1]:
import libspn as spn
import tensorflow as tf

### Building a Test Graph with Random Weights

In [2]:
iv_x = spn.IVs(num_vars=2, num_vals=2, name="iv_x")
sum_11 = spn.Sum((iv_x, [0,1]), name="sum_11")
sum_12 = spn.Sum((iv_x, [0,1]), name="sum_12")
sum_21 = spn.Sum((iv_x, [2,3]), name="sum_21")
sum_22 = spn.Sum((iv_x, [2,3]), name="sum_22")
prod_1 = spn.Product(sum_11, sum_21, name="prod_1")
prod_2 = spn.Product(sum_11, sum_22, name="prod_2")
prod_3 = spn.Product(sum_12, sum_22, name="prod_3")
root = spn.Sum(prod_1, prod_2, prod_3, name="root")
iv_y = root.generate_ivs(name="iv_y")
spn.generate_weights(root, init_value=spn.ValueType.RANDOM_UNIFORM(0, 1))

### Visualizing the SPN Graph

In [3]:
spn.display_spn_graph(root)

### Specify Training Data

In [4]:
iv_x_arr=[[0,0],[0,0],[1,1],[1,1],[1,1],[0,1],[0,1],[0,1]]
iv_y_arr=[[-1]] * len(iv_x_arr)

### Add Learning Ops

In [8]:
init_weights = spn.initialize_weights(root)
learning = spn.EMLearning(root, initial_accum_value=2)
init_learning = learning.reset_accumulators()
accumulate_updates = learning.accumulate_updates()
update_spn = learning.update_spn()
likelihood = tf.reduce_mean(learning.value.values[root])

### Run Learning

In [9]:
epoch = 0
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    while run():
        likelihood_arr, _ = sess.run([likelihood, accumulate_updates],
                                  feed_dict={iv_x:iv_x_arr, iv_y:iv_y_arr})
        print("Avg. Likelihood: %s" % (likelihood_arr))
        sess.run(update_spn)
        epoch+=1
        if epoch > 10:
            break

Avg. Likelihood: -1.31961
Avg. Likelihood: -1.26135
Avg. Likelihood: -1.24154
Avg. Likelihood: -1.23465
Avg. Likelihood: -1.23136
Avg. Likelihood: -1.22951
Avg. Likelihood: -1.22834
Avg. Likelihood: -1.22755
Avg. Likelihood: -1.22698
Avg. Likelihood: -1.22656
Avg. Likelihood: -1.22623


In [7]:
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    try:
        while run():
            likelihoods, _ =  sess.run([likelihood, accumulate_updates])
            print("Avg. Likelihood: %s" % (avg_likelihood))
            sess.run(update_spn)
            
    except tf.errors.OutOfRangeError:
        print("TRAINING DONE!")

InvalidArgumentError: You must feed a value for placeholder tensor 'iv_x/Placeholder' with dtype int32
	 [[Node: iv_x/Placeholder = Placeholder[dtype=DT_INT32, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'iv_x/Placeholder', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/czikus/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/czikus/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/czikus/.local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/czikus/.local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File 